In [49]:
import pdfplumber
from pdfminer.high_level import extract_text

import io
import requests
import re

import pandas as pd

In [19]:
doc_parlement = "https://www.lachambre.be/FLWB/PDF/55/3197/55K3197001.pdf" 
doc2 = "http://www.lachambre.be/FLWB/PDF/55/3509/55K3509001.pdf"
doc3 = "http://www.lachambre.be/FLWB/PDF/55/2894/55K2894001.pdf"
doc4 = "http://www.lachambre.be/FLWB/PDF/55/2092/55K2092001.pdf"
plenary_doc = "" # 3- Séances Plénières integral - Maciej
com_integral = "" # 4- Commissions integral - Mourad
bulletin = "" # 5- Bulletins des questions et réponses écrites - Ramina
rapport = "" # 6- Rapports déposés en exécution d'une loi - Sarkis

In [131]:
def get_text(url):
    rq = requests.get(url)
    left_column_text_all_pages = ""
    right_column_text_all_pages = ""

    with pdfplumber.open(io.BytesIO(rq.content)) as pdf:
        for page in pdf.pages:
            words = page.extract_words()
            threshold_x = 300

            left_column_words = [word for word in words if word['x0'] < threshold_x]
            right_column_words = [word for word in words if word['x0'] >= threshold_x]

            left_column_text = ' '.join([word['text'] for word in left_column_words])
            right_column_text = ' '.join([word['text'] for word in right_column_words])

            left_column_text_all_pages += left_column_text + ' '
            right_column_text_all_pages += right_column_text + ' '

            left_column_clean = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', left_column_text_all_pages) # remove end_line hyphens
            left_column_clean = re.sub(r'DOC 55 \d{4}/001', '', left_column_clean)
            left_column_clean = re.sub(r'\d{4}/001 DOC 55', '', left_column_clean)
            left_column_clean = re.sub(r'CHAMBRE \d+e SESSION DE LA 55e LÉGISLATURE \d{4}(?: \d+)?', '', left_column_clean)

            right_column_clean = re.sub(r'(?<=\w)-(\s)(?=\w)', r'', right_column_text_all_pages) # remove end_line hyphens
            right_column_clean = re.sub(r'DOC 55 \d{4}/001', '', right_column_clean)
            right_column_clean = re.sub(r'\d{4}/001 DOC 55', '', right_column_clean)
            right_column_clean = re.sub(r'\d{4} KAMER • \d+e ZITTING VAN DE 55e ZITTINGSPERIODE(?: \d+)?', '', right_column_clean)

    return left_column_clean, right_column_clean

    # {"fr_pdf_text" : left_column_text_all_pages, "nl_pdf_text": right_column_text_all_pages}
    

In [118]:
# left_column_clean, right_column_clean = get_text(doc_parlement)
get_text(doc_parlement)[1]

In [171]:
df = pd.read_json('apercu_complet.json').head(10)
df_pdf_not_disponible = df[df["DocumentChambre"].str.contains("pasdisponible")]
df_pdf_disponible = df[~df["DocumentChambre"].str.contains("pasdisponible")]

In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3505 entries, 0 to 3504
Data columns (total 98 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   first_title                    3505 non-null   object 
 1   document_number                3505 non-null   int64  
 2   document_page_url              3505 non-null   object 
 3   main_title                     3505 non-null   object 
 4                                  3505 non-null   object 
 5   Documentsprincipaux            3505 non-null   object 
 6   DocumentChambre                3505 non-null   object 
 7   Datededépôt                    2546 non-null   object 
 8   Typededocument                 3505 non-null   object 
 9   Datededistribution             3501 non-null   object 
 10  Dated'envoi                    2980 non-null   object 
 11  Auteur(s)                      3500 non-null   object 
 12  COMMISSIONCHAMBRE              1574 non-null   o

In [ ]:
# df_pdf_not_disponible["fr_text"] = "PDF not disponible"
# df_pdf_not_disponible["nl_text"] = "PDF not disponible"
df_pdf_disponible["fr_text"] = df_pdf_disponible["DocumentChambre"].map(lambda x: get_text(x)[0])
df_pdf_disponible["nl_text"] = df_pdf_disponible["DocumentChambre"].map(lambda x: get_text(x)[1])

In [172]:
df = pd.read_json('apercu_complet.json')

def add_text_to_df(df):
    df_pdf_not_disponible = df[df["DocumentChambre"].str.contains("pasdisponible")]
    df_pdf_disponible = df[~df["DocumentChambre"].str.contains("pasdisponible")]
    df_pdf_disponible["fr_text"] = df_pdf_disponible["DocumentChambre"].map(lambda x: get_text(x)[0])
    df_pdf_disponible["nl_text"] = df_pdf_disponible["DocumentChambre"].map(lambda x: get_text(x)[1])
    df_pdf_not_disponible["fr_text"] = "PDF not disponible"
    df_pdf_not_disponible["nl_text"] = "PDF not disponible"

In [ ]:
# def sent_to_words(sentences):
#     for sentence in sentences:
#         yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))